<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir ~/.kaggle
from google.colab import files
files.upload()
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle/

!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip walmart-recruiting-store-sales-forecasting
!unzip '*.csv.zip'
!unzip '*.csv.zip'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


Saving kaggle.json to kaggle.json
total 4
-rw------- 1 root root 71 Jul  6 18:13 kaggle.json
walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  walmart-recruiting-store-sales-forecasting.zip
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv.zip        
replace sampleSubmission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv.zip  
replace stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: stores.csv              
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv.zip            
replace train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv.zip           
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ena

In [2]:

!pip install prophet
!pip install -q dagshub mlflow

import dagshub
dagshub.init(repo_owner='eghib22', repo_name='Store-Sales-Forecasting', mlflow=True)

import mlflow

Accessing as agasi22

Initialized MLflow to track repo "eghib22/Store-Sales-Forecasting"

Repository eghib22/Store-Sales-Forecasting initialized!

In [3]:
import logging
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

from prophet import Prophet
from sklearn.metrics import mean_squared_error

train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

df = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
df = df.merge(stores, on='Store', how='left')
df = df.sort_values('Date')

In [4]:
for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
    df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

In [5]:

def weighted_mae(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


In [6]:

import logging
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)

results = []
all_preds = []

store_dept_groups = df.groupby(['Store', 'Dept'])
total_groups = len(store_dept_groups)

print(f"--- Starting Prophet Modeling for {total_groups} Store-Dept combos ---")

mlflow.set_experiment("Prophet_Forecasting")

with mlflow.start_run(run_name="Prophet_StoreDept_Forecast"):
    mlflow.log_param("model", "Prophet")
    mlflow.log_param("seasonality_yearly", True)
    mlflow.log_param("seasonality_weekly", True)
    mlflow.log_param("seasonality_daily", False)

    for idx, ((store_id, dept_id), group) in enumerate(store_dept_groups, start=1):
        print(f"\n--- Processing Store: {store_id}, Department: {dept_id} ({idx}/{total_groups}) ---")

        g = group.sort_values('Date').copy()
        g['ds'] = g['Date']
        g['y'] = g['Weekly_Sales']
        weights = g['IsHoliday'].apply(lambda x: 5 if x else 1)

        y_train = g[g['ds'] < '2012-01-01']
        y_val = g[(g['ds'] >= '2012-01-01') & (g['ds'] < '2012-07-01')]
        weights_val = weights.loc[y_val.index]

        if len(y_train) < 100 or len(y_val) < 20:
            print(f"   Skipping (Store {store_id}, Dept {dept_id}): Not enough data ({len(y_train)} train, {len(y_val)} val).")
            continue

        try:
            model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False
            )

            model.fit(y_train[['ds', 'y']])

            future = y_val[['ds']].copy()
            forecast = model.predict(future)

            y_pred = forecast['yhat'].values
            y_true = y_val['y'].values

            wmae = weighted_mae(y_true, y_pred, weights_val)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))

            print(f"   WMAE: {wmae:.2f}")

            results.append({
                'Store': store_id,
                'Dept': dept_id,
                'RMSE': rmse,
                'WMAE': wmae
            })

            all_preds.append(pd.DataFrame({
                'Date': y_val['ds'].values,
                'Store': store_id,
                'Dept': dept_id,
                'y_true': y_true,
                'y_pred': y_pred,
                'weight': weights_val.values
            }))

        except Exception as e:
            print(f"   Failed for Store {store_id}, Dept {dept_id}: {e}")
            continue

        gc.collect()

    print("\n--- Prophet Modeling Complete ---")

    if len(all_preds) > 0:
        all_df = pd.concat(all_preds)
        overall_wmae = np.sum(all_df['weight'] * np.abs(all_df['y_true'] - all_df['y_pred'])) / np.sum(all_df['weight'])
        print(f"Overall WMAE: {overall_wmae:.2f}")

        mlflow.log_metric("Overall_WMAE", overall_wmae)
        mlflow.log_metric("Total_StoreDept_Models", len(results))
    else:
        print("No valid predictions to aggregate!")

    results_df = pd.DataFrame(results)
    print(results_df.head())

    # Save CSV to your Drive
    results_df.to_csv('/content/drive/MyDrive/prophet_results.csv', index=False)

print("\n Done — results saved & run logged to MLflow.")

--- Starting Prophet Modeling for 3331 Store-Dept combos ---


DEBUG:cmdstanpy:input tempfile: /tmp/tmpemth1h52/oex9b2pp.json



--- Processing Store: 1, Department: 1 (1/3331) ---


DEBUG:cmdstanpy:input tempfile: /tmp/tmpemth1h52/y3tw_ypm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12730', 'data', 'file=/tmp/tmpemth1h52/oex9b2pp.json', 'init=/tmp/tmpemth1h52/y3tw_ypm.json', 'output', 'file=/tmp/tmpemth1h52/prophet_model8wuefnur/prophet_model-20250706181417.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:14:17 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:14:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


   WMAE: 4774.69


...


--- Processing Store: 45, Department: 93 (3326/3331) ---
   WMAE: 389.20


DEBUG:cmdstanpy:input tempfile: /tmp/tmpemth1h52/6jm08gk8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpemth1h52/5xl9jkwz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=6842', 'data', 'file=/tmp/tmpemth1h52/6jm08gk8.json', 'init=/tmp/tmpemth1h52/5xl9jkwz.json', 'output', 'file=/tmp/tmpemth1h52/prophet_modelvvwab6ww/prophet_model-20250706182948.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:29:48 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:29:48 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 45, Department: 94 (3327/3331) ---
   Skipping (Store 45, Dept 94): Not enough data (91 train, 26 val).

--- Processing Store: 45, Department: 95 (3328/3331) ---
   WMAE: 5721.23


DEBUG:cmdstanpy:input tempfile: /tmp/tmpemth1h52/tkt5izwh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpemth1h52/un2o0ce_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=7531', 'data', 'file=/tmp/tmpemth1h52/tkt5izwh.json', 'init=/tmp/tmpemth1h52/un2o0ce_.json', 'output', 'file=/tmp/tmpemth1h52/prophet_model_bku256k/prophet_model-20250706182949.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:29:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:29:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 45, Department: 96 (3329/3331) ---
   Skipping (Store 45, Dept 96): Not enough data (0 train, 1 val).

--- Processing Store: 45, Department: 97 (3330/3331) ---
   WMAE: 486.36

--- Processing Store: 45, Department: 98 (3331/3331) ---
   Skipping (Store 45, Dept 98): Not enough data (92 train, 26 val).

--- Prophet Modeling Complete ---
Overall WMAE: 1916.38
   Store  Dept         RMSE         WMAE
0      1     1  7593.523894  4774.693544
1      1     2  2949.514949  2379.769672
2      1     3  1389.780780  1227.278611
3      1     4  3300.410919  2598.559989
4      1     5  4799.037605  3460.748384
🏃 View run Prophet_StoreDept_Forecast at: https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow/#/experiments/2/runs/6ccf8db622e84f609b5fa9e7d28d2932
🧪 View experiment at: https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow/#/experiments/2

 Done — results saved & run logged to MLflow.


In [7]:
mlflow.end_run()